In [8]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [9]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [10]:
%sql sqlite:///my_data1.db

In [12]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

In [17]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
(sqlite3.OperationalError) table SPACEXTABLE already exists
[SQL: create table SPACEXTABLE as select * from SPACEXTBL where Date is not null]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [25]:
con = sqlite3.connect("my_data1.db")
query = 'SELECT DISTINCT "Launch_Site" FROM SPACEXTABLE;'
unique_sites = pd.read_sql_query(query, con)

In [26]:
print(unique_sites)

    Launch_Site
0   CCAFS LC-40
1   VAFB SLC-4E
2    KSC LC-39A
3  CCAFS SLC-40


In [33]:
query = '''
SELECT *
FROM SPACEXTABLE
WHERE "Launch_Site" LIKE 'KSC%'
LIMIT 5;
'''
df_ksc = pd.read_sql_query(query, con)
print(df_ksc)

         Date Time (UTC) Booster_Version Launch_Site        Payload  \
0  2017-02-19   14:39:00   F9 FT B1031.1  KSC LC-39A  SpaceX CRS-10   
1  2017-03-16    6:00:00     F9 FT B1030  KSC LC-39A    EchoStar 23   
2  2017-03-30   22:27:00  F9 FT  B1021.2  KSC LC-39A         SES-10   
3  2017-05-01   11:15:00   F9 FT B1032.1  KSC LC-39A        NROL-76   
4  2017-05-15   23:21:00     F9 FT B1034  KSC LC-39A  Inmarsat-5 F4   

   PAYLOAD_MASS__KG_      Orbit    Customer Mission_Outcome  \
0               2490  LEO (ISS)  NASA (CRS)         Success   
1               5600        GTO    EchoStar         Success   
2               5300        GTO         SES         Success   
3               5300        LEO         NRO         Success   
4               6070        GTO    Inmarsat         Success   

        Landing_Outcome  
0  Success (ground pad)  
1            No attempt  
2  Success (drone ship)  
3  Success (ground pad)  
4            No attempt  


In [34]:
query = '''
SELECT SUM("Payload_Mass__kg_") AS total_payload_kg
FROM SPACEXTABLE
WHERE "Customer" = 'NASA (CRS)';
'''

df_payload = pd.read_sql_query(query, con)
print(df_payload)

   total_payload_kg
0             45596


In [36]:
query = '''
SELECT AVG("Payload_Mass__kg_") AS avg_payload_kg
FROM SPACEXTABLE
WHERE "Booster_Version" = 'F9 v1.1';
'''

df_avg_payload = pd.read_sql_query(query, con)
print(df_avg_payload)

   avg_payload_kg
0          2928.4


In [37]:
query = '''
SELECT MIN("Date") AS first_successful_landing
FROM SPACEXTABLE
WHERE "Landing_Outcome" = 'Success (drone ship)';
'''

df_first_landing = pd.read_sql_query(query, con)
print(df_first_landing)

  first_successful_landing
0               2016-04-08


In [38]:
query = '''
SELECT DISTINCT "Booster_Version"
FROM SPACEXTABLE
WHERE "Landing_Outcome" = 'Success (ground pad)'
  AND "Payload_Mass__kg_" > 4000
  AND "Payload_Mass__kg_" < 6000;
'''

df_boosters = pd.read_sql_query(query, con)
print(df_boosters)

  Booster_Version
0   F9 FT B1032.1
1   F9 B4 B1040.1
2   F9 B4 B1043.1


In [39]:
query = '''
SELECT "Class" AS mission_outcome, COUNT(*) AS total
FROM SPACEXTABLE
GROUP BY "Class";
'''

df_mission_counts = pd.read_sql_query(query, con)
print(df_mission_counts)

  mission_outcome  total
0           Class    101


In [40]:
query = '''
SELECT "Booster_Version"
FROM SPACEXTABLE
WHERE "Payload_Mass__kg_" = (
    SELECT MAX("Payload_Mass__kg_")
    FROM SPACEXTABLE
);
'''

df_max_payload_boosters = pd.read_sql_query(query, con)
print(df_max_payload_boosters)

   Booster_Version
0    F9 B5 B1048.4
1    F9 B5 B1049.4
2    F9 B5 B1051.3
3    F9 B5 B1056.4
4    F9 B5 B1048.5
5    F9 B5 B1051.4
6    F9 B5 B1049.5
7   F9 B5 B1060.2 
8   F9 B5 B1058.3 
9    F9 B5 B1051.6
10   F9 B5 B1060.3
11  F9 B5 B1049.7 


In [41]:
query = '''
SELECT 
    substr("Date", 6, 2) AS month,
    "Landing_Outcome",
    "Booster_Version",
    "Launch_Site"
FROM SPACEXTABLE
WHERE "Landing_Outcome" = 'Success (ground pad)'
  AND substr("Date", 1, 4) = '2017';
'''

df_2017_ground_success = pd.read_sql_query(query, con)
print(df_2017_ground_success)

  month       Landing_Outcome Booster_Version   Launch_Site
0    02  Success (ground pad)   F9 FT B1031.1    KSC LC-39A
1    05  Success (ground pad)   F9 FT B1032.1    KSC LC-39A
2    06  Success (ground pad)   F9 FT B1035.1    KSC LC-39A
3    08  Success (ground pad)   F9 B4 B1039.1    KSC LC-39A
4    09  Success (ground pad)   F9 B4 B1040.1    KSC LC-39A
5    12  Success (ground pad)  F9 FT  B1035.2  CCAFS SLC-40


In [42]:
query = '''
SELECT "Landing_Outcome", COUNT(*) AS outcome_count
FROM SPACEXTABLE
WHERE "Date" BETWEEN '2010-06-04' AND '2017-03-20'
GROUP BY "Landing_Outcome"
ORDER BY outcome_count DESC;
'''

df_landing_counts = pd.read_sql_query(query, con)
print(df_landing_counts)

          Landing_Outcome  outcome_count
0              No attempt             10
1    Success (drone ship)              5
2    Failure (drone ship)              5
3    Success (ground pad)              3
4      Controlled (ocean)              3
5    Uncontrolled (ocean)              2
6     Failure (parachute)              2
7  Precluded (drone ship)              1
